In [8]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as ks

In [2]:
def LDA(D, Y=None, noOfDims=None):
    # initializing the variables
    summ, lenn = 0, 0
    # computing the mean for each class
    mu = []
    for x in D: 
        mu.append(np.mean(x, axis=0, keepdims=True).T)
        # calculating the sum and overall number
        # elements for overall mu
        summ += sum(x)
        lenn += len(x)
    mu = np.array(mu)
    overallMu = summ / lenn
    # computing the between class scatter matrices
    SB = 0
    for i in range(len(mu)): SB += len(D[i]) * np.matmul((mu[i]-overallMu), (mu[i]-overallMu).T)
    # centering the class matrices
    Z = np.array([D[i] - mu[i].T for i in range(len(D))])
    # computing the within class scatter matrices
    S = np.array([np.matmul(Z[i].T, Z[i]) for i in range(len(Z))])
    # performing eigenvalue decomposition
    eigValues, eigVectors = np.linalg.eigh(np.matmul(np.linalg.inv(sum(S)), SB))
    # returning the dominant eigen vectors
    return eigVectors[::-1][:noOfDims].T

In [3]:
def projectData(X_train, X_test, W):
    X_train = np.array([np.matmul(W.T, X_train[i]) for i in range(X_train.shape[0])])
    X_test = np.array([np.matmul(W.T, X_test[i]) for i in range(X_test.shape[0])])
    return X_train, X_test

In [65]:
def KMC(X, k, epsilon=0.01, maxIter=100):
  # initializing the error to be max at first
  err = float('inf')
  # randomly initializing the centroids
  centroid = [X[np.random.randint(0, len(X)-1)] for i in range(k)]
  while maxIter > 0 and err >= epsilon:
    maxIter -= 1
    cluster = [[] for i in range(k)]
    # copying the old centroids
    oldCentroid = copy.deepcopy(centroid)
    # assigning each data point to closest cluster
    for x in X:
      minn, j = float('inf'), 0
      for i in range(len(centroid)):
        dist = np.linalg.norm(x-centroid[i]) ** 2
        if dist < minn: minn, j = dist, i
      cluster[j].append(x)
    # updating the centroids
    for i in range(len(cluster)):
      if len(cluster[i]) != 0: centroid[i] = sum(cluster[i]) / len(cluster[i])
    # if centroids didn't change, then terminate
    err = 0
    for i in range(len(centroid)):
      err += np.linalg.norm(centroid[i] - oldCentroid[i]) ** 2
  return cluster